# <center>Определение стоимости автомобилей</center>

## <center>Введение</center>

Целью проекта является построение модели для определения рыночной стоимости автомобилей. Датасет содержит дату скачивания анкеты из базы данных по продажам `DateCrawled`, тип кузова `VehicleType`;
+ год регистрации автомобиля `RegistrationYear`
+ месяц регистрации автомобиля ` RegistrationMonth`

<br>тип коробки передач `Gearbox`, мощность (л. с.) `Power`, модель `Model`, пробег в км `Kilometer `, тип топлива `FuelType`, марка автомобиля `Brand`, была машина в ремонте или нет `NotRepaired`, дата создания анкеты `DataCreated`, количество фотографий автомобиля `NumberOfPictures`, почтовый индекс владельца автомобиля `PostalCode` и дата последней активности пользователя `LastSeen`.

<br>**Целевой признак**, то есть цена автомобиля в евро - `Price`.

## <center>План проекта</center>

<br>**Импорт библиотек**
<br>**Предобработка данных**
<br>**"Тупая" модель**
<br>**Модели sklearn:**
+ Линейная регрессия
+ Дерево решений
+ Случайный лес

<br>**LightGBM**
<br>**Вывод**

## <center>Импорт библиотек</center>

In [1]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_error
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

#!pip install lightgbm
from lightgbm import LGBMRegressor

## <center>Предобработка данных<center>

Считаем данные и посмотрим на них:

In [2]:
data = pd.read_csv('autos.csv')

display(data.head(20))
print(data.info())

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21
5,2016-04-04 17:36:23,650,sedan,1995,manual,102,3er,150000,10,petrol,bmw,yes,2016-04-04 00:00:00,0,33775,2016-04-06 19:17:07
6,2016-04-01 20:48:51,2200,convertible,2004,manual,109,2_reihe,150000,8,petrol,peugeot,no,2016-04-01 00:00:00,0,67112,2016-04-05 18:18:39
7,2016-03-21 18:54:38,0,sedan,1980,manual,50,other,40000,7,petrol,volkswagen,no,2016-03-21 00:00:00,0,19348,2016-03-25 16:47:58
8,2016-04-04 23:42:13,14500,bus,2014,manual,125,c_max,30000,8,petrol,ford,NaN,2016-04-04 00:00:00,0,94505,2016-04-04 23:42:13
9,2016-03-17 10:53:50,999,small,1998,manual,101,golf,150000,0,NaN,volkswagen,NaN,2016-03-17 00:00:00,0,27472,2016-03-31 17:17:06


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

Проблем много: и пропуски, и типы - начнём с самого простого. Уберём всё то, что нам не нужно.

Наша задача - построить модель, которая определяет рыночную стоимость автомобиля по его характеристикам, то есть в нашем датасете лишние **очевидно**:

+ `DateCrawled` — дата скачивания анкеты из базы, причём здесь она :)
+ `LastSeen` — дата последней активности пользователя, она здесь тоже ни при чём :)

Дальнейшие решения более рискованные, однако если наша цель - построить модель, вычисляющую стоимость **чисто по характеристикам**, то так же лишние:

+ `PostalCode` — почтовый индекс владельца анкеты (пользователя). Вообще местонахождение продавца может оказывать некое влияние на стоимость - однако это должно отражаться иначе, быть категориальным признаком. Превращать этот столбец в категорию "местонахождение" - большая головная боль, плюс местонахождение продавца технической характеристикой автомобиля **не** является.
+ `NumberOfPictures` — количество фотографий автомобиля. Вообще презентация так же может оказывать определённое влияние, однако количество фоточек тоже НЕ является технической характеристикой автомобиля;
+ `DateCreated` — дата создания анкеты. Влияние может оказывать, но снова не техническая характеристика;
+ `RegistrationMonth` — месяц регистрации автомобиля. **Год** явно имеет большое значение, старые машины (полагаю) стоят дешевле, чем новые (либо в некоторых случаях наоборот, дороже, ибо винтаж такой винтаж), однако **месяц** - история странная, от неё, пожалуй, можно избавиться.

Итого:

In [3]:
data = data.drop(['DateCrawled', 'LastSeen', 'PostalCode', 'NumberOfPictures', 
                  'DateCreated', 'RegistrationMonth'], axis=1)

display(data.head(20))
print(data.info())

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,FuelType,Brand,NotRepaired
0,480,NaN,1993,manual,0,golf,150000,petrol,volkswagen,NaN
1,18300,coupe,2011,manual,190,NaN,125000,gasoline,audi,yes
2,9800,suv,2004,auto,163,grand,125000,gasoline,jeep,NaN
3,1500,small,2001,manual,75,golf,150000,petrol,volkswagen,no
4,3600,small,2008,manual,69,fabia,90000,gasoline,skoda,no
5,650,sedan,1995,manual,102,3er,150000,petrol,bmw,yes
6,2200,convertible,2004,manual,109,2_reihe,150000,petrol,peugeot,no
7,0,sedan,1980,manual,50,other,40000,petrol,volkswagen,no
8,14500,bus,2014,manual,125,c_max,30000,petrol,ford,NaN
9,999,small,1998,manual,101,golf,150000,NaN,volkswagen,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Price             354369 non-null  int64 
 1   VehicleType       316879 non-null  object
 2   RegistrationYear  354369 non-null  int64 
 3   Gearbox           334536 non-null  object
 4   Power             354369 non-null  int64 
 5   Model             334664 non-null  object
 6   Kilometer         354369 non-null  int64 
 7   FuelType          321474 non-null  object
 8   Brand             354369 non-null  object
 9   NotRepaired       283215 non-null  object
dtypes: int64(4), object(6)
memory usage: 27.0+ MB
None


Так-то лучше :) Вопрос вызывает 7-ая строка - с ценой в ноль. Посмотрим, сколько у нас таких записей:

In [4]:
print(len(data[data['Price'] == 0]))

10772


Вообще не так много, можно было бы и выбросить, цена 0 очень похожа на ошибку... Но возможно, ошибкой не является - запомним, что такое есть, и оставим. И попробуем разобраться с пропусками.

Логично предположить, что если данных о ремонте нет, то скорее всего машина в ремонте **не** была, то есть `NotRepaired` - это `yes`. Далее:

In [5]:
print(data['Gearbox'].value_counts())
print(data['FuelType'].value_counts())

manual    268251
auto       66285
Name: Gearbox, dtype: int64
petrol      216352
gasoline     98720
lpg           5310
cng            565
hybrid         233
other          204
electric        90
Name: FuelType, dtype: int64


Ручная коробка передач встречается чаще автоматической, её можно взять "значением по умолчанию" - как и бензин. 

In [6]:
print(data['VehicleType'].value_counts())
print(data['Model'].value_counts())

sedan          91457
small          79831
wagon          65166
bus            28775
convertible    20203
coupe          16163
suv            11996
other           3288
Name: VehicleType, dtype: int64
golf                  29232
other                 24421
3er                   19761
polo                  13066
corsa                 12570
                      ...  
i3                        8
serie_3                   4
rangerover                4
range_rover_evoque        2
serie_1                   2
Name: Model, Length: 250, dtype: int64


С типом кузова и моделью гадать на кофейной гуще как-то вообще глупо, наверное - введём новое значение `NA`, означающее "не указано". Это, наверное, создаст определённые проблемы при обучении, но что делать - выкидывать 30к записей жалко.

Заполнение вручную реализуем на **копии** сета: 

In [7]:
data_manual = data.copy()

data_manual['Gearbox'] = data_manual['Gearbox'].fillna('manual')
data_manual['FuelType'] = data_manual['FuelType'].fillna('petrol')
data_manual['NotRepaired'] = data_manual['NotRepaired'].fillna('yes')
data_manual['VehicleType'] = data_manual['VehicleType'].fillna('NA')
data_manual['Model'] = data_manual['Model'].fillna('NA')

display(data_manual.head(20))
print(data_manual.info())

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,FuelType,Brand,NotRepaired
0,480,NA,1993,manual,0,golf,150000,petrol,volkswagen,yes
1,18300,coupe,2011,manual,190,NA,125000,gasoline,audi,yes
2,9800,suv,2004,auto,163,grand,125000,gasoline,jeep,yes
3,1500,small,2001,manual,75,golf,150000,petrol,volkswagen,no
4,3600,small,2008,manual,69,fabia,90000,gasoline,skoda,no
5,650,sedan,1995,manual,102,3er,150000,petrol,bmw,yes
6,2200,convertible,2004,manual,109,2_reihe,150000,petrol,peugeot,no
7,0,sedan,1980,manual,50,other,40000,petrol,volkswagen,no
8,14500,bus,2014,manual,125,c_max,30000,petrol,ford,yes
9,999,small,1998,manual,101,golf,150000,petrol,volkswagen,yes


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Price             354369 non-null  int64 
 1   VehicleType       354369 non-null  object
 2   RegistrationYear  354369 non-null  int64 
 3   Gearbox           354369 non-null  object
 4   Power             354369 non-null  int64 
 5   Model             354369 non-null  object
 6   Kilometer         354369 non-null  int64 
 7   FuelType          354369 non-null  object
 8   Brand             354369 non-null  object
 9   NotRepaired       354369 non-null  object
dtypes: int64(4), object(6)
memory usage: 27.0+ MB
None


Замечательно - как и то, что проблема с неправильными типами решилась сама собой, их больше нет. Запомним, что здесь у нас численное, а что - категориальное (OHE будет сделано дальше в пайплайне):

In [8]:
numeric_manual = ['RegistrationYear', 'Power', 'Kilometer']
categorial_manual = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']

Разделим на признаки и целевой:

In [9]:
features = data_manual.drop(['Price'], axis=1)
target = data_manual['Price']

И определимся с точкой отсчёта.

## <center>"Тупая" модель</center>

Создадим два пайплайна: предобработку, которая пригодится нам и в дальнейшем, и реализацию тупой модели. Предобработка:

In [10]:
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_manual),
        ('cat', categorical_transformer, categorial_manual)])

"Тупая" модель:

In [11]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42) 

dummy_pipeline = make_pipeline(preprocessor, DummyRegressor(strategy='median'))
dummy_pipeline.fit(X_train, y_train)
predicted_train = dummy_pipeline.predict(X_train)
predicted_test = dummy_pipeline.predict(X_test)

dummy_rmse_train = mean_squared_error(y_train, predicted_train) ** 0.5
dummy_rmse_test = mean_squared_error(y_test, predicted_test) ** 0.5
print(f'RMSE, медиана: обучающая - {dummy_rmse_train: .4f}; тестовая - {dummy_rmse_test: .4f}')

RMSE, медиана: обучающая -  4817.5782; тестовая -  4877.1056


In [12]:
dummy_pipeline = make_pipeline(preprocessor, DummyRegressor(strategy='mean'))
dummy_pipeline.fit(X_train, y_train)
predicted_train = dummy_pipeline.predict(X_train)
predicted_test = dummy_pipeline.predict(X_test)

dummy_rmse_train = mean_squared_error(y_train, predicted_train) ** 0.5
dummy_rmse_test = mean_squared_error(y_test, predicted_test) ** 0.5
print(f'RMSE, среднее: обучающая - {dummy_rmse_train: .4f}; тестовая - {dummy_rmse_test: .4f}')

RMSE, среднее: обучающая -  4505.0852; тестовая -  4550.2920


Точка отсчёта ясна, попробуем сделать лучше.

## <center>Модели sklearn</center>

### <center>Линейная регрессия</center>

Начнём с самой обычной линейной регрессии:

In [13]:
lr_pipeline = make_pipeline(preprocessor, LinearRegression())

In [14]:
%%time
lr_pipeline.fit(X_train, y_train)

Wall time: 6.57 s


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['RegistrationYear', 'Power',
                                                   'Kilometer']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['VehicleType', 'Gearbox',
                                                   'Model', 'FuelType', 'Brand',
                                                   'NotRepaired'])])),
                ('linearregression', LinearRegression())])

In [15]:
%%time
predicted_train = lr_pipeline.predict(X_train)

Wall time: 479 ms


In [16]:
%%time
predicted_test = lr_pipeline.predict(X_test)

Wall time: 116 ms


In [17]:
lr_rmse_train = mean_squared_error(y_train, predicted_train) ** 0.5
lr_rmse_test = mean_squared_error(y_test, predicted_test) ** 0.5
print(f'RMSE: обучающая - {lr_rmse_train: .4f}; тестовая - {lr_rmse_test: .4f}')

RMSE: обучающая -  3168.6234; тестовая -  3226.1512


Не очень качественно, зато быстро: локально `Wall time` обучения - 6.89 s, а предсказания - 475 ms.

## <center>Дерево решений</center>

Посмотрим, как справится дерево. Здесь уже будем производить поиск по сетке и без функции RMSE не обойтись:

In [18]:
def rmse_score(y_true, y_predicted):
    temp = (y_true - y_predicted) ** 2
    return (temp.sum() / len(y_true)) ** 0.5

In [19]:
#прогон долгий, поэтому код закомменчен :)

#tree_pipeline = Pipeline(steps=[('prep', preprocessor), 
#                                ('est', DecisionTreeRegressor(random_state=42))])
#tree_param = {'est__max_depth': [2, 3, 4, 5, 6, 7, 8, 9, 10]}
#
#gs_tree = GridSearchCV(tree_pipeline,
#                      param_grid=tree_param,
#                      scoring=make_scorer(rmse_score, greater_is_better=False),
#                      cv=5, verbose=0)
#gs_tree.fit(X_train, y_train)

#print(gs_tree.best_score_)
#print(gs_tree.best_params_)

`-2124.649080071005
{'est__max_depth': 10}`. Максимум в 10, поэтому смотрим дальше:

In [20]:
#tree_param = {'est__max_depth': [11, 12, 13, 14, 15, 16, 17, 18, 19, 20]}

#gs_tree = GridSearchCV(tree_pipeline,
#                      param_grid=tree_param,
#                      scoring=make_scorer(rmse_score, greater_is_better=False),
#                      cv=5, verbose=0)
#gs_tree.fit(X_train, y_train)

#print(gs_tree.best_score_)
#print(gs_tree.best_params_)

Результат: `-2032.0784768324706
{'est__max_depth': 15}`. То есть дерево с глубиной 15 - посмотрим, как оно справляется с тестовой выборкой:

In [21]:
tree_pipeline = Pipeline(steps=[('prep', preprocessor), 
                                ('est', DecisionTreeRegressor(max_depth=15, random_state=42))])

In [22]:
%%time
tree_pipeline.fit(X_train, y_train)

Wall time: 14.6 s


Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['RegistrationYear', 'Power',
                                                   'Kilometer']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['VehicleType', 'Gearbox',
                                                   'Model', 'FuelType', 'Brand',
                                                   'NotRepaired'])])),
                ('est', DecisionTreeRegressor(max_depth=15, random_state=42))])

In [23]:
%%time
predicted_test = tree_pipeline.predict(X_test)

Wall time: 132 ms


In [24]:
tree_rmse_test = mean_squared_error(y_test, predicted_test) ** 0.5
print(f'RMSE тестовая - {tree_rmse_test: .4f}')

RMSE тестовая -  2044.9767


Со временем обучения здесь всё, конечно, уже хуже: `Wall time` - 12.6 s, предсказание - 128 ms. Однако всё ещё неплохо и результат симпатичнее. Однако "плюс-минус две тыщи евро" - это всё же не очень хорошо, попробуем сделать лучше.

### <center>Случайный лес</center>

Начнём с грубой прикидки:

In [25]:
#forest_pipeline = Pipeline(steps=[('prep', preprocessor), 
#                                ('est', RandomForestRegressor(random_state=42))])
#forest_param = [{'est__max_depth': [5, 10, 15],
#              'est__n_estimators': [5, 10, 20]}]

#gs_forest = GridSearchCV(forest_pipeline,
#                      param_grid=forest_param,
#                      scoring=make_scorer(rmse_score, greater_is_better=False),
#                      cv=5, verbose=0)
#gs_forest.fit(X_train, y_train)

#print(gs_forest.best_score_)
#print(gs_forest.best_params_)

Результат: `-1854.948631257735
{'est__max_depth': 15, 'est__n_estimators': 20}`. Чуть получше, чем дерево, увеличим количество деревьев:

In [26]:
#forest_pipeline = Pipeline(steps=[('prep', preprocessor), 
#                                ('est', RandomForestRegressor(random_state=42))])
#forest_param = [{'est__max_depth': [5, 10, 15],
#              'est__n_estimators': [25, 30]}]

#gs_forest = GridSearchCV(forest_pipeline,
#                      param_grid=forest_param,
#                      scoring=make_scorer(rmse_score, greater_is_better=False),
#                      cv=5, verbose=0)
#gs_forest.fit(X_train, y_train)

#print(gs_forest.best_score_)
#print(gs_forest.best_params_)

Результат: `-1850.6191146085728
{'est__max_depth': 15, 'est__n_estimators': 30}`. Разница совсем не велика, остановимся на 20:

In [27]:
forest_pipeline = Pipeline(steps=[('prep', preprocessor), 
                                ('est', RandomForestRegressor(max_depth=15, n_estimators=20, random_state=42))])

In [28]:
#%%time
#forest_pipeline.fit(X_train, y_train)

#это три минуты, так что закоммент 

In [29]:
#%%time
#predicted_test = forest_pipeline.predict(X_test)

In [30]:
#tree_rmse_test = mean_squared_error(y_test, predicted_test) ** 0.5
#print(f'RMSE тестовая - {tree_rmse_test: .4f}')

3 минуты на обучение (`Wall time: 3min 22s`) - это, конечно, грустно, и выигрыш не то чтоб сильно большой: от ~2000 к ~1850 (`RMSE тестовая -  1861.9861`). Что ж, мы сделали всё, что могли - перейдём к градиентному бустингу.

## <center>LightGBM</center>

Вернёмся к оригинальному датасету - обозначим для бустинга категориальные признаки, и пусть он справляется с ними сам, как хочет:

In [31]:
for feature in categorial_manual:
    data[feature] = pd.Series(data[feature], dtype='category')
    
features = data.drop(['Price'], axis=1)
target = data['Price']
    
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42) 

Начнём со 100 деревьев:

In [32]:
model = LGBMRegressor(boosting_type='gbdt', max_depth=15, n_estimators=100, random_state=42)

model.fit(X_train, y_train)
predicted_test = model.predict(X_test)
rmse = rmse_score(predicted_test, y_test)
print(f'RMSE; градиентный бустинг: {rmse: .4f}')

RMSE; градиентный бустинг:  1790.4564


А вот это - хороший результат, причём полученный очень быстро:

In [33]:
%%time
model.fit(X_train, y_train)

Wall time: 881 ms


LGBMRegressor(max_depth=15, random_state=42)

In [34]:
%%time
predicted_test = model.predict(X_test)

Wall time: 250 ms


Бустинг справляется меньше, чем за секунду (`Wall time: 850 ms`) - попробуем аккуратно подобрать параметры:

In [35]:
#model = LGBMRegressor(boosting_type='gbdt', random_state=42)

#params = [{'max_depth': [12, 13, 14, 15, 16, 17],
#           'n_estimators': [20, 40, 60, 80, 100, 120, 140, 160, 180, 200], 
#           'learning_rate': [0.025, 0.05, 0.075, 0.1, 0.15, 0.2, 0.25, 0.3]}]

#gs_boost = GridSearchCV(model,
#                      param_grid=params,
#                      scoring=make_scorer(rmse_score, greater_is_better=False),
#                      cv=5, verbose=0)
#gs_boost.fit(X_train, y_train)

#print(gs_boost.best_score_)
#print(gs_boost.best_params_)

Результат: `-1725.3042835426743
{'learning_rate': 0.3, 'max_depth': 17, 'n_estimators': 200}`. Попробуем пойти дальше:

In [36]:
#model = LGBMRegressor(boosting_type='gbdt', random_state=42)

#params = [{'max_depth': [15, 16, 17, 18, 19, 20],
#           'n_estimators': [200, 250, 300, 350, 400], 
#           'learning_rate': [0.2, 0.3, 0.4]}]

#gs_boost = GridSearchCV(model,
#                      param_grid=params,
#                      scoring=make_scorer(rmse_score, greater_is_better=False),
#                      cv=5, verbose=0)
#gs_boost.fit(X_train, y_train)

#print(gs_boost.best_score_)
#print(gs_boost.best_params_)

Результат: `-1707.3579470552186
{'learning_rate': 0.2, 'max_depth': 15, 'n_estimators': 400}`. На самом деле, я бы сказала, что ковырять дальше нет смысла - выигрыш очень небольшой, а деревьев и так уже много. Мы будем увеличивать время работы модели ради почти ничего. Но ради любопытства можно посмотреть и дальше:

In [37]:
#model = LGBMRegressor(boosting_type='gbdt', random_state=42)

#params = [{'max_depth': [13, 14, 15, 16, 17, 18],
#           'n_estimators': [400, 500, 600], 
#           'learning_rate': [0.1, 0.15, 0.2, 0.3]}]

#gs_boost = GridSearchCV(model,
#                      param_grid=params,
#                      scoring=make_scorer(rmse_score, greater_is_better=False),
#                      cv=5, verbose=0)
#gs_boost.fit(X_train, y_train)

#print(gs_boost.best_score_)
#print(gs_boost.best_params_)

Результат: `-1697.8620878244099
{'learning_rate': 0.15, 'max_depth': 17, 'n_estimators': 600}`. Да, разница всё меньше. Остановимся на исходной модели `{'learning_rate': 0.3, 'max_depth': 17, 'n_estimators': 200}`:

In [38]:
model = LGBMRegressor(boosting_type='gbdt', max_depth=17, n_estimators=200, learning_rate=0.3, random_state=42)

In [39]:
%%time
model.fit(X_train, y_train)

Wall time: 1.08 s


LGBMRegressor(learning_rate=0.3, max_depth=17, n_estimators=200,
              random_state=42)

In [40]:
%%time
predicted_test = model.predict(X_test)

Wall time: 305 ms


In [41]:
rmse = rmse_score(predicted_test, y_test)
print(f'RMSE; градиентный бустинг: {rmse: .4f}')

RMSE; градиентный бустинг:  1733.0255


~1700 при обучении в секунду (`Wall time: 1.16 s`) - это, конечно, сильно, даже линейная регрессия с её ужасным результатом не обучалась так быстро.

## <center>Вывод</center>

Сводная таблица моделей:

In [42]:
models = {'Линейная регрессия (sklearn)':['Высокая (6 с)', 'Высокая (100 мс)', 'Низкое (3200)'],
          'Дерево решений (sklearn)':['Низкая (13 c)', 'Высокая (140 мс)', 'Среднее (2050)'], 
          'Случайный лес (sklearn)':['Очень низкая (3 м)', 'Средняя (350 мс)', 'Высокое (1850)'],
         'Градиентный бустинг (LightGBM)':['Очень высокая (1 с)', 'Средняя (300 мс)', 'Высокое (1750)'],}
df_models = pd.DataFrame.from_dict(models, orient='index', 
                                   columns=['Скорость обучения', 'Скорость предсказания', 'Качество'])

display(df_models)

,Скорость обучения,Скорость предсказания,Качество
Линейная регрессия (sklearn),Высокая (6 с),Высокая (100 мс),Низкое (3200)
Дерево решений (sklearn),Низкая (13 c),Высокая (140 мс),Среднее (2050)
Случайный лес (sklearn),Очень низкая (3 м),Средняя (350 мс),Высокое (1850)
Градиентный бустинг (LightGBM),Очень высокая (1 с),Средняя (300 мс),Высокое (1750)


Бустинг LightGBM выигрывает по всем статьям, кроме скорости предсказания - но, на мой взгляд, это не та разница, ради которой стоит брать дерево и заметно жертвовать точностью и скоростью обучения. Скорость предсказания всё равно не велика, это не минуты и даже не секунды - так что останавливаемся на нём.